In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients

In [2]:
# load model
device='cuda'
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base",num_labels=3)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
def oh(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [3]:
ENTUBE = 'entube.parquet'
entube = pd.read_parquet(ENTUBE)
entube=entube[["title","label_2"]].rename({'title': 'text', 'label_2': 'label'}, axis=1)

In [4]:
from datasets import Dataset
dataset=Dataset.from_pandas(entube)
dataset=dataset.shuffle(seed=42)
dataset[0]

{'text': '[ Bình Luận CF ] Khi thay cận chiến bằng 6 Nòng VIP - Tiền Zombie v4',
 'label': 2}

In [5]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets=tokenized_datasets.train_test_split(test_size=0.1,seed=123)
tokenized_datasets

Map:   0%|          | 0/26236 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 23612
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2624
    })
})

In [6]:
small_train_dataset = tokenized_datasets['train']
small_eval_dataset  = tokenized_datasets['test']

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
import evaluate
metric = evaluate.load("f1", average='micro')

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels , average='micro')

In [10]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir='test_trainer',
    do_predict= True, 
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps = 1000,
    save_steps = 1000,
    save_total_limit = 2,
    load_best_model_at_end=True,)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

C:\Users\mtthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/8856 [00:00<?, ?it/s]

{'loss': 0.9723, 'learning_rate': 4.717705510388438e-05, 'epoch': 0.17}
{'loss': 1.0813, 'learning_rate': 4.435411020776874e-05, 'epoch': 0.34}


  0%|          | 0/328 [00:00<?, ?it/s]

{'eval_loss': 0.9187567830085754, 'eval_f1': 0.20274390243902438, 'eval_runtime': 65.9082, 'eval_samples_per_second': 39.813, 'eval_steps_per_second': 4.977, 'epoch': 0.34}
{'loss': 0.912, 'learning_rate': 4.153116531165312e-05, 'epoch': 0.51}
{'loss': 0.8612, 'learning_rate': 3.870822041553749e-05, 'epoch': 0.68}


...

  0%|          | 0/328 [00:00<?, ?it/s]

{'eval_loss': 0.8395326137542725, 'eval_f1': 0.14557926829268292, 'eval_runtime': 85.2268, 'eval_samples_per_second': 30.788, 'eval_steps_per_second': 3.849, 'epoch': 2.37}
{'loss': 0.658, 'learning_rate': 7.655826558265583e-06, 'epoch': 2.54}
{'loss': 0.6456, 'learning_rate': 4.832881662149955e-06, 'epoch': 2.71}


  0%|          | 0/328 [00:00<?, ?it/s]

{'eval_loss': 0.8203840255737305, 'eval_f1': 0.15510670731707318, 'eval_runtime': 70.6343, 'eval_samples_per_second': 37.149, 'eval_steps_per_second': 4.644, 'epoch': 2.71}
{'loss': 0.6491, 'learning_rate': 2.009936766034327e-06, 'epoch': 2.88}
{'train_runtime': 4951.282, 'train_samples_per_second': 14.307, 'train_steps_per_second': 1.789, 'train_loss': 0.7801847311431692, 'epoch': 3.0}


TrainOutput(global_step=8856, training_loss=0.7801847311431692, metrics={'train_runtime': 4951.282, 'train_samples_per_second': 14.307, 'train_steps_per_second': 1.789, 'train_loss': 0.7801847311431692, 'epoch': 3.0})

In [13]:
a=trainer.predict(small_eval_dataset)

  0%|          | 0/328 [00:00<?, ?it/s]

In [17]:
predict=np.argmax(a[0], axis=1)
labels=a[1]

In [19]:
from sklearn.metrics import classification_report
print(classification_report(labels, predict))

              precision    recall  f1-score   support

           0       0.70      0.66      0.68       940
           1       0.43      0.56      0.49       768
           2       0.78      0.63      0.70       916

    accuracy                           0.62      2624
   macro avg       0.64      0.62      0.62      2624
weighted avg       0.65      0.62      0.63      2624



In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import torch
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [28]:
# load model
model = AutoModelForSequenceClassification.from_pretrained('test_trainer/checkpoint-5000')
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
def predict(inputs):
    return model(inputs)[0]

In [30]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [31]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [33]:
def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0]

In [35]:
lig = LayerIntegratedGradients(custom_forward, model.roberta.embeddings)

In [269]:
def explain(i):
    text = small_eval_dataset[i]['text']
    label = small_eval_dataset[i]['label']
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    attributions, delta = lig.attribute(inputs=input_ids,
                                        baselines=ref_input_ids,
                                        n_steps=700,
                                        internal_batch_size=3,
                                        return_convergence_delta=True)
    score = predict(input_ids)
    def summarize_attributions(attributions):
        attributions = attributions.sum(dim=-1).squeeze(0)
        attributions = attributions / torch.norm(attributions)
        return attributions
    attributions_sum = summarize_attributions(attributions)
    # storing couple samples in an array for visualization purposes
    score_vis = viz.VisualizationDataRecord(attributions_sum,
                                            torch.softmax(score, dim = 1)[0][torch.argmax(torch.softmax(score, dim = 1)[0])],
                                            torch.argmax(torch.softmax(score, dim = 1)[0]),
                                            label,
                                            text,
                                            attributions_sum.sum(),       
                                            all_tokens,
                                            delta)
    print('\033[1m', 'Visualization For Score', '\033[0m')
    print(viz.visualize_text([score_vis]))

In [270]:
for i in range(20):
    explain(i)

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (0.87),Mate 30 Pro KHÔNG CÓ GOOGLE: đừng lo,2.03,#s Mate 30 Pro KHÔNG C@@ Ó G@@ OO@@ G@@ LE@@ : đừng lo #/s


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


...

<IPython.core.display.HTML object>
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.44),Ảo ảnh thị giác | 3D Illusion | Vannie in Wonderland | THẾ GIỚI MA THUẬT,-0.39,#s Ả@@ o ảnh thị giác | 3D Illu@@ sion | Van@@ nie in Won@@ derland | TH@@ Ế GI@@ ỚI MA THU@@ ẬT #/s


<IPython.core.display.HTML object>
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (0.80),7 mẫu smartphone THẤT BẠI của năm 2020: có cả iPhone!,3.17,#s 7 mẫu smartphone TH@@ ẤT B@@ ẠI của năm 20@@ 20@@ : có cả i@@ Phone@@ ! #/s


<IPython.core.display.HTML object>
